In [ ]:
!wget https://colab.research.google.com/drive/1Te4m1xpq7WJnqhM9NVHivIqfLGFAEgfu#scrollTo=-yS4xZjkr82R
from IPython.display import Image
Image(filename='image.png')

In [28]:
!apt install openmpi-bin openmpi-common libopenmpi-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopenmpi-dev is already the newest version (4.1.2-2ubuntu1).
openmpi-bin is already the newest version (4.1.2-2ubuntu1).
openmpi-common is already the newest version (4.1.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [18]:
%%writefile Scatter_Gather.c
#include "mpi.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define PI 3.14159265358979323846

int main(int argc, char **argv) {
    int myid, numprocs;
    int n = 1000000;
    double local_sum = 0.0, global_sum = 0.0;
    double factor;
    int local_n, start, end;

    MPI_Init(&argc, &argv);
    MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
    MPI_Comm_rank(MPI_COMM_WORLD, &myid);

    local_n = n / numprocs;
    start = myid * local_n;
    end = start + local_n;

    if (myid == numprocs - 1) {
        end = n;
    }


    for (int i = start; i < end; i++) {
        factor = (i % 2 == 0) ? 1.0 : -1.0;
        local_sum += factor / (2 * i + 1);
    }


    double *all_sums = NULL;
    if (myid == 0) {
        all_sums = (double *)malloc(numprocs * sizeof(double));
    }
    MPI_Gather(&local_sum, 1, MPI_DOUBLE, all_sums, 1, MPI_DOUBLE, 0, MPI_COMM_WORLD);

    if (myid == 0) {
        global_sum = 0.0;
        for (int i = 0; i < numprocs; i++) {
            global_sum += all_sums[i];
        }
        global_sum *= 4.0;
        printf("Calculated Pi: %.16f\n", global_sum);
        free(all_sums);
    }

    MPI_Finalize();
    return 0;
}


Overwriting Scatter_Gather.c


In [19]:
!mpicc -o Scatter_Gather Scatter_Gather.c

In [23]:
!mpiexec  --allow-run-as-root --oversubscribe -n 2 ./Scatter_Gather
!mpiexec  --allow-run-as-root --oversubscribe -n 4 ./Scatter_Gather
!mpiexec  --allow-run-as-root --oversubscribe -n 8 ./Scatter_Gather

Calculated Pi: 3.1415916535896922
Calculated Pi: 3.1415916535897810
Calculated Pi: 3.1415916535897277


In [25]:
%%writefile Send_Receive.c
#include "mpi.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define PI 3.14159265358979323846

int main(int argc, char **argv) {
    int myid, numprocs;
    int n = 1000000;
    double local_sum = 0.0, global_sum = 0.0;
    double factor;
    int local_n, start, end;

    MPI_Init(&argc, &argv);
    MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
    MPI_Comm_rank(MPI_COMM_WORLD, &myid);

    local_n = n / numprocs;
    start = myid * local_n;
    end = start + local_n;

    if (myid == numprocs - 1) {
        end = n;
    }

    for (int i = start; i < end; i++) {
        factor = (i % 2 == 0) ? 1.0 : -1.0;
        local_sum += factor / (2 * i + 1);
    }

    if (myid != 0) {
        MPI_Send(&local_sum, 1, MPI_DOUBLE, 0, 0, MPI_COMM_WORLD);
    } else {
        global_sum = local_sum;
        for (int i = 1; i < numprocs; i++) {
            MPI_Recv(&local_sum, 1, MPI_DOUBLE, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            global_sum += local_sum;
        }
        global_sum *= 4.0;
        printf("Calculated Pi: %.16f\n", global_sum);
    }

    MPI_Finalize();
    return 0;
}


Writing Send_Receive.c


In [26]:
!mpicc -o Send_Receive Send_Receive.c

In [27]:
!mpiexec  --allow-run-as-root --oversubscribe -n 2 ./Send_Receive
!mpiexec  --allow-run-as-root --oversubscribe -n 4 ./Send_Receive
!mpiexec  --allow-run-as-root --oversubscribe -n 8 ./Send_Receive

Calculated Pi: 3.1415916535896922
Calculated Pi: 3.1415916535897810
Calculated Pi: 3.1415916535897277
